# PaperPi Structure
Supervisor will loop and poll each plugin for an update. Plugins are all`Plugin()` objects with a `poll` method

When polled `Plugin()` objects respond with a structured list (tuple/dict) with the following information:
* Priority(`int`): 0 (high) -- 10+ (low); plugins with negative values are excluded
* Update Required(`bool`): True update; False do not update

Plugin objects require the following parameters:
* configuration(`dict`) argparse/config.ini style dict
* resolution (this may be pulled from the configuration?)
* name(`str`) human readable name for logging and reference
* update_function(`func`): function that will run and provide updates and status

# To Do
* [ ] item
## Runtime
* [ ] figure out how to import my_help and get all the functions from within the main plugin
* [ ] systemctl scripts 
* [ ] systemctl testing

## Plugins
* [ ] weather -- yr weather

## Distribution
* [ ] develop .spec for pyinstaller
* [ ] packaging scripts
* [ ] documentation
* [ ] item

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import logging.config
import shutil
import sys
from itertools import cycle
from inspect import getfullargspec
from importlib import import_module
from time import sleep
from pathlib import Path
from os import chdir
from distutils.util import strtobool

In [3]:
import ArgConfigParse
from epdlib import Screen
from epdlib.Screen import Update
from library.CacheFiles import CacheFiles
from library.Plugin import Plugin
from library.InterruptHandler import InterruptHandler

import constants
import errors

In [4]:
def do_exit(status=0, message=None):
    if message:
        if status > 0:
            logging.error(f'unexpected exit: {message}')
        border = '\n'+'#'*70 + '\n'
        message = border + message + border + '\n***Exiting***'
        print(message)
        
    try:
        sys.exit(status)
    except Exception as e:
        pass

In [5]:
def get_cmd_line_args():
    cmd_args = ArgConfigParse.CmdArgs()
    cmd_args.add_argument('-c', '-config', ignore_none=True, metavar='CONFIG_FILE.ini',
                         type=str, dest='user_config',
                         help='use the specified configuration file')
    
    cmd_args.add_argument('-l', '--log_level', ignore_none=True, metavar='LOG_LEVEL',
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
                         dest='main__log_level', help='change the log output level')
    
    cmd_args.add_argument('-m', '--run_module', required=False, default=None,
                         ignore_none=True)
    
    cmd_args.add_argument('-d', '--daemon', required=False, default=False,
                         dest='main__daemon', action='store_true', 
                         help='run in daemon mode (ignore user configuration if found)')
    
    cmd_args.add_argument('-V', '--version', required=False, default=False, ignore_false=True,
                          action='store_true',
                          help='display version and exit')
    
    

   
    cmd_args.parse_args()
    
 
    return cmd_args
    
    

In [17]:
def run_module_function(plugin=None):
    if not plugin:
        print(f'General plugin information: $ {constants.app_name.lower()} -m module')
        print(f'Run a specific plugin function: {constants.app_name.lower()} -m module.function')
        return
        
    plugin = plugin.split('.')
    module = plugin[0]
    
    if len(plugin) > 1:
        function = plugin[1]
    else:
        function = None

    try:
        my_plugin = import_module(f'plugins.{module}.{module}')
    except ModuleNotFoundError as e:
        print(f'Plugin "{module}" not found. Check the name and try again')
        print('##### known plugins #####')
        for i in Path('./plugins').glob('*'):
            if i.is_dir() and i.name[0] not in ('_', '.'):
                print(i.name)     
        return
    
    if function:
        getattr(my_plugin, function)()
    else:
        try:
            my_plugin.my_help.my_help()
        except AttributeError:
            print(f'module "{my_plugin.__name__}" has no documentation or user-facing functions at this time')


In [30]:
run_module_function('lms_client.scan_servers')

Scanning for available LMS Server and players
servers found:
[{'host': '192.168.178.9', 'port': 9000}]
players found:
name: slimpi
playerid: dc:a6:32:29:99:f0
modelname: SqueezeLite


players found:
name: MacPlay
playerid: 68:5b:35:b5:97:bf
modelname: SqueezePlay




In [ ]:
def get_config_files(cmd_args):
    '''read config.ini style file(s)
    Args:
        cmd_args(`ArgConfigParse.CmdArgs()` object)
    
    Returns:
        `ArgConfigParse.ConfigFile`'''
    config_files_list = [constants.config_base, constants.config_system, constants.config_user]
    
    daemon = False
    
    if hasattr(cmd_args, "main__daemon"):
        if cmd_args.main_daemon:
            logging.debug('excluding user config files')
            config_files_list.pop()
            daemon = True
        else:
            daemon = False
    else: 
        daemon = False
    
    if not daemon:
        # create a user config directory
        if not constants.config_user.exists():
            logging.info('creating user config directory and inserting config file')
            try:
                constants.config_user.parent.mkdir(parents=True, exist_ok=True)
            except PermissionError as e:
                logging.warning(f'could not create {constants.config_user}: {e}')
        if not constants.config_user.exists():
            try:
                shutil.copy(constants.config_base, constants.config_user)
            except Exception as e:
                do_exit(1, f'could not copy configuratoin file to {constants.config_user}: {e}')
                
        

    
    config_files = ArgConfigParse.ConfigFile(config_files_list, ignore_missing=True)
    config_files.parse_config()
                    
    
    return config_files

In [ ]:
def convert(d, func, exceptions):
    for section, values in d.items():
        for key, value in values.items():
            try:
                sanitized = func(value)
            except exceptions:
                sanitized = value
            d[section][key] = sanitized
    return d

In [ ]:
def sanitize_vals(config):
    '''attempt to convert all the strings into appropriate formats
        integer/float like strings ('7', '100', '-1.3') -> int
        boolean like strings (yes, no, Y, t, f, on, off) -> 0 or 1
    Args:
        config(`dict`): nested config.ini style dictionary
    
    Returns:
        `dict`'''
    
    def convert(d, func, exceptions):
        '''convert values in nested dictionary using a specified function
            values that raise an exception are left unchanged
        
        Args:
            d(`dict`): nested dictionary {'Section': {'key': 'value'}}
            func(`function`): function such as int() or strtobool()
            exceptions(`tuple`): 
            
        Returns:
            `dict`'''
        for section, values in d.items():
            for key, value in values.items():
                try:
                    sanitized = func(value)
                except exceptions:
                    sanitized = value
                d[section][key] = sanitized
        return d
    
    #str
    str_to_ints = convert(config, int, (ValueError))
    str_to_bool = convert(str_to_ints, strtobool, (ValueError, AttributeError))
    
    config = str_to_bool
    
    
        
    return config

In [ ]:
def setup_splash(config, resolution):
    logging.debug('checking splash settings')
    if 'splash' in config['main']:
        logging.debug('checking splash screen settings')
        if config['main']['splash']:
            splash = True
        else:
            logging
            splash = False
    else:
        splash = True

    if splash:
        from plugins.splash_screen import splash_screen
        splash_config = { 
            'name': 'Splash Screen',
            'layout': splash_screen.layout.layout,
            'update_function': splash_screen.update_function,
            'resolution': resolution
        } 
        logging.debug('configuring and displaying splash screen')
        splash = Plugin(**splash_config)
        splash.update(constants.app_name, constants.version, constants.url)    
    return splash

In [ ]:
def setup_display(config):
    try:
        logging.debug('setting display type')
        epd_module = '.'.join([constants.waveshare_epd, config['main']['display_type']])
        epd = import_module(epd_module)
    except KeyError as e:
        do_exit(1, errors.keyError_fmt.format('main', 'display_type'))
    except ModuleNotFoundError as e:
        logging.error('Check your config files and ensure a known waveshare_epd display is specified!')
        do_exit(1, errors.moduleNotFoundError_fmt.format(config["main"]["display_type"], e))
        
    screen = Screen()
    try:
        screen.epd = epd
    except PermissionError as e:
        logging.critical(f'Error initializing EPD: {e}')
        logging.critical(f'The user executing {constants.app_name} does not have access to the SPI device.')
        do_exit(1, 'This user does not have access to the SPI group\nThis can typically be resolved by running:\n$ sudo groupadd <username> spi')

    try:
        config['main']['rotation'] = int(config['main']['rotation'])
    except KeyError as e:
        logging.info(errors.keyError_fmt.format('main', 'rotation'))
        logging.info('using default: 0')
    try:
        screen.rotation = config['main']['rotation']
    except ValueError as e:
        logging.error('invalid rotation; valid values are: 0, 90, -90, 180')
        do_exit(1, errors.keyError_fmt.format('main', 'rotation'))
        
    return screen

In [ ]:
def build_plugin_list(config, resolution, cache):
    # get the expected key-word args from the Plugin() spec
    spec_kwargs = getfullargspec(Plugin).args

    plugins = []

    for section, values in config.items():
        # ignore the other sections
        if section.startswith('Plugin:'):
            logging.info(f'[[ {section} ]]')

            my_config = {}
            # add all the spec_kwargs from the config
            plugin_kwargs = {}
            for key, val in values.items():
                if key in spec_kwargs:
                    my_config[key] = val
                else:
                    # add everything that is not one of the spec_kwargs to this dict
                    plugin_kwargs[key] = val

            # populate the kwargs my_config dict that will be passed to the Plugin() object
            my_config['name'] = section
            my_config['resolution'] = resolution
            my_config['config'] = plugin_kwargs
            my_config['cache'] = cache
            try:
                module = import_module(f'{constants.plugins}.{values["plugin"]}')
                my_config['update_function'] = module.update_function
                my_config['layout'] = getattr(module.layout, values['layout'])
            except KeyError as e:
                ## FIX ME -- do a graceful exit here?
                logging.info('no module specified; skipping update_function and layout')
            my_plugin = Plugin(**my_config)
            try:
                my_plugin.update()
            except AttributeError:
                ## FIX ME -- do a graceful exit here?
                logging.info('skipping update due to missing update_function')
            logging.info(f'appending plugin')
            plugins.append(my_plugin)
    return plugins

In [ ]:
def update_loop(plugins, screen):
    # use itertools cycle to move between list elements
    plugin_cycle = cycle(plugins)
    plugin_is_active = False
    # current plugin for display
    this_plugin = next(plugin_cycle)
    # track time plugin is displayed for
    this_plugin_timer = Update()
    max_priority = -1
    # each plugin generates a unique hash whenever it is updated
    this_hash = ''

    logging.info('starting update loop')
    with InterruptHandler() as h:
        while True:    
            if h.interrupted:
                logging.info('caught interrupt -- stoping execution')
                break
            logging.info('updating plugins')
            priority_list = []

            for plugin in plugins:
                logging.debug(f'[[ {plugin.name} ]]')
                plugin.update()
                # record the priority of all active plugins (priority < 0 is considered inactive)
                if plugin.priority >= 0:
                    priority_list.append(plugin.priority)

            # priority increases as it approaches 0; negative priorities are considered inactive        
            max_priority = min(priority_list)
            logging.info(f'current maximum priority: {max_priority}')

            logging.info(f'displaying plugins')
            logging.info(f'[[ {this_plugin.name} ]]')

            if this_plugin_timer.last_updated > this_plugin.min_display_time:
                logging.info(f'this plugin\'s display timer expired; switching plugin')
                plugin_is_active = False

                while not plugin_is_active:
                    this_plugin = next(plugin_cycle)
                    logging.debug(f'checking priority of {plugin.name}')
                    if this_plugin.priority >= 0 and this_plugin.priority <= max_priority:
                        plugin_is_active = True
                    else:
                        logging.debug(f'priority is too low: {this_plugin.priority}, max_priority: {max_priority}')
                        plugin_is_active = False                
                        ## this would be a good place to track failed attempts to find a plugin with appropriate
                        ## display value

                logging.debug(f'preparing to display {this_plugin.name}')

                if this_hash != this_plugin.hash:
                    logging.debug('plugin image has refreshed -- refreshing screen')
                    this_hash = this_plugin.hash
                    screen.initEPD()
                    screen.writeEPD(this_plugin.image)
                else:
                    logging.debug('plugin image has not refreshed -- skiping screen refresh')
                this_plugin_timer.update()

            ## FIX ME -- find a better way to sleep here?
            sleep(1)    

In [ ]:
def main():
    # set the absolute path to the current directory
    absolute_path = constants.absolute_path
    
    # change directory into the current working directory
    # to simplify futher path related work
    chdir(absolute_path)
    
    # set up logging
    logging.config.fileConfig(constants.logging_config)
    logger = logging.getLogger(__name__)
    
    # get command line and config file arguments
    cmd_args = get_cmd_line_args()
    config_files = get_config_files(cmd_args)
    
    # merge file and commandline (right-most over-writes left)
    config = ArgConfigParse.merge_dict(config_files.config_dict, cmd_args.nested_opts_dict)
    
    if cmd_args.options.version:
        print(constants.version_string)
        return
    
    if cmd_args.options.run_module:
        run_module_function(cmd_args.options.run_module)
        return
        
    
    # make sure all the integer-like strings are converted into integers
    config = sanitize_vals(config)
    
    logger.setLevel(config['main']['log_level'])
    screen = setup_display(config)
    splash = setup_splash(config, screen.resolution)    
    screen.initEPD()
    if splash:
        screen.writeEPD(splash.image)
        
    
    cache = CacheFiles(path_prefix=constants.app_name)
    plugins = build_plugin_list(config, screen.resolution, cache)
    
    update_loop(plugins, screen)

    logging.warning('caught terminate signal -- cleaning up and exiting')
    cache.cleanup()
    screen.initEPD()
    screen.clearEPD()
    

In [ ]:
c = main()

In [ ]:
sys.argv

In [ ]:
sys.argv.pop()


In [ ]:
sys.argv.append('-m')
sys.argv.append('basic_clock')

In [ ]:
sys.argv.append('lms_client')

In [ ]:
sys.argv.append('open_weather')